# chap 4
## Limitations of Option
*Option* doesn't tell us anything about what went wrong in the case of an exceptional condition.

A simple extension to Option is the Either data type, which let us track ...

## Either Type
---
```scala
sealed trait Either[+E, +A]
case class Left[+E] (value: E) extends Either[E, Nothing]
case class Right[+A](value: A) extends Either[Nothing, A]
```
---

By convention
- *Right* constructor is reserved for the success case(a pun on "right," meaning correct), and
- *Left* is used for failure. Note the suggestive name **E** (for Error)

### Usage Example of Either
```scala
def mean(xs: Seq[Double]): Either[String, Double] =
  if (xs.isEmpty)
    Left("mean of empty list!")
  else
    Right(xs.sum / xs.length)
    
def safeDiv(x: Int, y: Int): Either[Exception, Int] =
  try Right(x/y)
  catch{ case e: Exception => Left(e) }
  
def Try[A](a: => A) : Either[Exception, A] =
  try Right(a)
  catch { catch e => Left(e) }

In [10]:
sealed trait Either[+E, +A] {
  def pure[E,A](a: A): Either[E,A] = Right(a)
  
  def map[B](f: A => B): Either[E, B] = this match {
    case Right(v) => Right(f(v))
    case l @ Left(_) => l
  }
  
  def flatMap[EE >: E, B](f: A => Either[EE, B]): Either[EE, B] = this match {
    case Right(v) => f(v)
    case l @ Left(_) => l
  }
  
  def orElse[EE >: E, B >: A](default: => Either[EE, B]): Either[EE, B] = this match {
    case Right(v) => this
    case _ => default
  }
  
//   def map2[EE >: E, B, C](fa: Either[E, A], fb: Either[E, B])(f: (A, B) => C) : Either[EE, C] = 
//     flatMap(a => fb.map(b => f(a, b)))
  def map2[EE >: E, B, C](fb: Either[EE, B])(f: (A, B) => C) : Either[EE, C] = 
    (this, fb) match {
      case (Right(a), Right(b)) => Right(f(a,b))
      case (Right(_), l @ Left(m)) => l
      case (l @ Left(m), _) => l
    }
           
  def sequence[E, A](es: List[Either[E, A]]): Either[E, List[A]] = 
    // traverse(es)(a => a)
    traverse(es)(identity)
  
//   def foldRight[A, B](z: B)(f: (A, B) => B): B = ???
  def traverse[E, A, B](as: List[A])(f: A => Either[E, B]): Either[E, List[B]] = ???
//     as.foldRight(pure[E,List[B]](List.empty[B]))((a: A, acc: Either[E, List[B]]) => map2(f(a),acc)(_ :: _))
//       as.foldRight(pure[E,List[B]](List.empty[B]): Either[E, List[B]])((a: A, acc: Either[E, List[b]]) => map2(f(a),acc)(_ :: _))
//       as.foldRight[Either[E, List[B]](pure[E,List[B]](List.empty[B]))((a: A, acc: Either[E, List[b]]) => map2(f(a),acc)(_ :: _))
}
case class Left[+E] (value: E) extends Either[E, Nothing]
case class Right[+A](value: A) extends Either[Nothing, A]

defined trait Either
defined class Left
defined class Right

In [8]:
val e1: Either[String, Int] = Right(5)
val r1 = e1.map(_ + 1)
val e2: Either[String, Int] = Left("hello")
val r2 = e2.map(_ + 2)

e1: Either[String, Int] = Right(5)
r1: Either[String, Int] = Right(6)
e2: Either[String, Int] = Left(hello)
r2: Either[String, Int] = Left(hello)

In [ ]:
def parseInsuranceRateQuote( age: )

In [ ]:
def sequence[E, A](es: List[Either[E, A]]): Either[E, List[A]] = ???

def traverse[E, A, B](as: List[A])(f: A => Either[E, B]): Either[E, List[B]] = ???

In [4]:
Left("abc").map(_ + 1)

cmd4.sc:1: value + is not a member of Nothing
val res4 = Left("abc").map(_ + 1)
                             ^

: 

In [6]:
Right(1).flatMap(a => Right(a + 10))

res5: Either[Nothing, Int] = Right(11)

# Homework
In this implementation, map2 is only able to report one error,
even if both the name and the age are invalid.
What would you need

# Strictness and Laziness
**Non-strictness** is a fundamental technique for improving on
- efficiency
- modularrity

of functional programs in general.

## Motivating Example
```scala
List(1,2,3,4).map(_ + 10).filter(_ % 2 == 0).map(_ * 3)
```